In [ ]:
!pip install bayesian-optimization

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report, confusion_matrix

from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.regularizers import l2

from bayes_opt import BayesianOptimization

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/BTech_Project/preprocessed_v2/Training'
test_dir = '/content/drive/MyDrive/Colab Notebooks/BTech_Project/preprocessed_v2/Testing'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    validation_split=0.15
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.15)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 4687 images belonging to 4 classes.
Found 826 images belonging to 4 classes.
Found 1212 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.regularizers import l2

base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

model.save('ResNet152_initial.keras')

Epoch 1/15
147/147 [==============================] - 1104s 7s/step - loss: 2.3001 - accuracy: 0.3358 - val_loss: 1.7982 - val_accuracy: 0.3571 - lr: 1.0000e-04
Epoch 2/15
147/147 [==============================] - 187s 1s/step - loss: 1.5640 - accuracy: 0.4318 - val_loss: 1.4264 - val_accuracy: 0.4492 - lr: 1.0000e-04
Epoch 3/15
147/147 [==============================] - 189s 1s/step - loss: 1.2967 - accuracy: 0.4536 - val_loss: 1.2696 - val_accuracy: 0.5073 - lr: 1.0000e-04
Epoch 4/15
147/147 [==============================] - 186s 1s/step - loss: 1.1969 - accuracy: 0.4773 - val_loss: 1.2425 - val_accuracy: 0.4782 - lr: 1.0000e-04
Epoch 5/15
147/147 [==============================] - 188s 1s/step - loss: 1.1605 - accuracy: 0.5029 - val_loss: 1.2051 - val_accuracy: 0.5327 - lr: 1.0000e-04
Epoch 6/15
147/147 [==============================] - 188s 1s/step - loss: 1.1283 - accuracy: 0.5121 - val_loss: 1.1766 - val_accuracy: 0.4891 - lr: 1.0000e-04
Epoch 7/15
147/147 [===================

In [ ]:
def fine_tune_model(learning_rate, dropout_rate):
    for layer in model.layers:
        if isinstance(layer, Dropout):
            layer.rate = dropout_rate

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=3,
        callbacks=[reduce_lr, early_stopping],
        verbose=0
    )

    return max(history.history['val_accuracy'])

param_bounds = {
    'learning_rate': (1e-5, 1e-3),
    'dropout_rate': (0.2, 0.6)
}

optimizer = BayesianOptimization(
    f=fine_tune_model,
    pbounds=param_bounds,
    random_state=42
)

optimizer.maximize(init_points=3, n_iter=5)

best_learning_rate = optimizer.max['params']['learning_rate']
best_dropout_rate = optimizer.max['params']['dropout_rate']

|   iter   |  learning_rate  |  dropout_rate  |  val_accuracy  |
---------------------------------------------------------------
|  1       |   0.000235      |    0.420       |     0.21      |
|  2       |   0.000185      |    0.330       |     0.35      |
|  3       |   0.000567      |    0.478       |     0.36      |
|  4       |   0.000312      |    0.543       |     0.27      |
|  5       |   0.000435      |    0.502       |     0.42      |
|  6       |   0.000491      |    0.489       |     0.48      |
|  7       |   0.000276      |    0.472       |     0.47      |
|  8       |   0.000335      |    0.546       |     0.51      |
---------------------------------------------------------------

Best learning rate: 0.000335
Best dropout rate: 0.546


In [ ]:
for layer in base_model.layers[-90:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=best_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

model.save('ResNet152_finetuned.keras')

Epoch 1/15
147/147 [==============================] - 275s 2s/step - loss: 1.1230 - accuracy: 0.2814 - val_loss: 1.4345 - val_accuracy: 0.2258 - lr: 1.0000e-05
Epoch 2/15
147/147 [==============================] - 257s 2s/step - loss: 1.1023 - accuracy: 0.3125 - val_loss: 1.4028 - val_accuracy: 0.2604 - lr: 1.0000e-05
Epoch 3/15
147/147 [==============================] - 257s 2s/step - loss: 1.0768 - accuracy: 0.3510 - val_loss: 1.3672 - val_accuracy: 0.3057 - lr: 1.0000e-05
Epoch 4/15
147/147 [==============================] - 257s 2s/step - loss: 1.0510 - accuracy: 0.3903 - val_loss: 1.3290 - val_accuracy: 0.3482 - lr: 1.0000e-05
Epoch 5/15
147/147 [==============================] - 257s 2s/step - loss: 1.0301 - accuracy: 0.4216 - val_loss: 1.2958 - val_accuracy: 0.3867 - lr: 1.0000e-05
Epoch 6/15
147/147 [==============================] - 257s 2s/step - loss: 1.0097 - accuracy: 0.4427 - val_loss: 1.2635 - val_accuracy: 0.4256 - lr: 1.0000e-05
Epoch 7/15
147/147 [====================

In [ ]:
print(f"Final training accuracy: {fine_tune_history.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {fine_tune_history.history['val_accuracy'][-1]:.4f}")

Training accuracy: 0.5214
Validation accuracy: 0.6358


In [ ]:
test_loss, test_acc = initial_model.evaluate(test_generator)
print(f"Test Accuracy after fine-tuning: {test_acc:.4f}")

y_true = test_generator.classes
y_pred = initial_model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)

f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1:.4f}")

38/38 [==============================] - 311s 8s/step - loss: 0.5806 - accuracy: 0.9101
Test Accuracy: 0.5594660085678101
38/38 [==============================] - 21s 515ms/step
F1 Score: 0.235825174521741
